# 4. **SeisIO.Quake Submodule**
The Quake submodule was created to work with earthquake data. It uses a new\
structure called `SeisEvent` that holds seismic source info, an event header,\
and trace data.\
\
This tutorial assumes basic familiarity with SeisIO. If you haven't done parts\
1–3 yet, do those first!

## A. **Installation**
If you haven't run this tutorial before as the current user,\
then please run this set of commands: (press Shift + Enter)

In [ ]:
using Pkg
Pkg.resolve()

## B. **Getting Started**

In [ ]:
using SeisIO, SeisIO.Quake
include("safe_rm.jl")

\
There are four basic parametric Types in the Quake submodule:\
* SeisHdr
* SeisSrc
* EventTraceData
* SeisEvent

### **SeisHdr**
Seismic header information, including location and magnitude data.

In [ ]:
?SeisHdr

\
A new event header object can be initialiazed with keywords:

In [ ]:
import Dates:DateTime
H1 = SeisHdr(ot=DateTime("2012-01-03T03:49:45"), int=(0x02, "MMI"))

\
Because EQMag and EQLoc are custom Types, these can also be initialized with keywords,\
even when creating a SeisHdr structure:

In [ ]:
H1 = SeisHdr(id="who_runs_this_network:9081278017348", ot=DateTime("2012-01-03T03:49:45"), int=(0x05, "MMI"), mag = EQMag(val=5.5f0, scale="mww", nst=94, gap=35.5), src = "tutorial", typ = "earthquake")

### **SeisSrc** 
Seismic source descriptions: focal mechanism, moment tensor, planes, axes, and source-time function.

In [ ]:
?SeisSrc

### **EventTraceData**
A similar container to the core `SeisData` structure, with additional fields for\
source-receiver geometry and arrival times. A single channel of EventTraceData\
is a custom type named `EventChannel`, analogous to a `SeisChannel` object.

In [ ]:
?EventTraceData

### **SeisEvent**
Finally, a `SeisEvent` structure has three substructures:
* Header (type `SeisHdr`)
* Seismic Source (type `SeisSrc`)
* Trace Data (type `EventTraceData`)

As before, one can initialize these at creation with keywords:

In [ ]:
using SeisIO.RandSeis
W = SeisEvent(hdr = SeisHdr(
        id="who_runs_this_network:9081278017348", 
        ot=DateTime("2012-01-03T03:49:45"), 
        int=(0x05, "MMI"), 
        mag = EQMag(val=5.5f0, scale="mww", nst=94, gap=35.5), 
        src = "tutorial", 
        typ = "earthquake"), 
    source = SeisSrc(m0 = 1.0e18,
        mt = randn(6).*1.0e18, 
        dm = randn(6).*1.0e17), 
    data = EventTraceData(randSeisData(6))
)

# Note: calling EventTraceData on a SeisData structure converts it.

In [ ]:
W.data

## C. **File I/O**

### **Legacy File Formats**
Legacy files can be read into memory in their entirety using the `read_quake`\
wrapper, which works like `read_data` for SeisEvent structures:

In [ ]:
?read_quake

In [ ]:
# one of the largest events at Mt. Hood before 2002 -- data from Jones & Malone, BSSA 2005
# for the UW format, we omit the trailing letter to read both header and data files

W = read_quake("uw", "DATA/99011116541")

\
**Caution**: not every legacy data format produces output whose fields contain\
usable data. In the above example, the UW file format doesn't fully store\
the principal axes or nodal planes; it records strike (*θ*) and dip (*δ*), but\
not rake (*λ*).

### **QuakeML**
Read QuakeML files into memory with either `read_qml(fname)` (to\
output arrays of header and source parameters) or\
`read_quake("qml", fname)` (for one event):

In [ ]:
qmf = "DATA/fdsnws-event_2017-01-12T03-18-55Z.xml"
(H,R) = read_qml(qmf)
W1 = SeisEvent(hdr = H[1], source = R[1])
W2 = read_quake("qml", qmf)

In [ ]:
W1 == W2

### **ASDF**

#### Reading QuakeML from ASDF
Just as waveform data can be read from ASDF volumes, so can QuakeML:

In [ ]:
using SeisIO.SeisHDF
hdf_evt = "DATA/example.h5"
(H,R) = asdf_rqml(hdf_evt)

\
The outputs of the above command are:
* **H**, an Array{SeisHdr,1} with preferred event header info.
* **R**, an Array{SeisSrc,1} with corresponding seismic source parameters.


`H[i]` and `R[i]` describe the same event for any `i`; thus, it should\
always be true that `H.id[i] == R.eid[i]`.\
\
To create a SeisEvent structure for the first event's header and source info,\
run the following command:

In [ ]:
W1 = SeisEvent(hdr = H[1], source = R[1])

#### Writing QuakeML to ASDF
The command `asdf_wqml(W)` writes the `:hdr` and `:source` fields of SeisEvent\
structure `W` to an ASDF volume. If the file exists, and already contains a\
"QuakeML" group, then data are appended to existing QuakeML. Otherwise, the\
"QuakeML" group is initialized with `W.hdr` and `W.source`.

In [ ]:
hdf_out = "temp.h5"
safe_rm(hdf_out)

# Create file and initialize "QuakeML" group
asdf_wqml(hdf_out, W1)

# Append to "QuakeML" group
asdf_wqml(hdf_out, W1)

# Read it back in
(H,R) = asdf_rqml(hdf_out)

\
Since the above command sequence wrote W1 to QuakeML twice...

In [ ]:
H[1] == H[2]

In [ ]:
R[1] == R[2]

## D. **Data Requests**

### **Event Info**
Use `FDSNevq` to query FDSN Event servers for information:

In [ ]:
?FDSNevq

In [ ]:
# This should return 9 events
(H,R) = FDSNevq("201103110547", mag=[3.0, 9.9], src="IRIS", v=0)

In [ ]:
length(H)

In [ ]:
# What's new at Mt. Rainier? Anything since the start of last month?
rr = Float64[46.852886, -121.760374, 0.0, 0.1]
mr = [-5.0, 9.9]

using Dates
ed = Dates.now()
mm = Month(ed)
sd = mm.value == 1 ? Dates.DateTime(Year(ed)-Year(1), Month(12)) : Dates.DateTime(Year(ed), Month(ed)-Month(1))
ds = div((ed-sd).value, 1000)

(H,R) = FDSNevq(string(sd), rad=rr, mag=mr, evw=[ds, 0.0], src="IRIS")

In [ ]:
length(H)

\
Using `src="all"` queries every FDSN Event server known to SeisIO, but beware:\
This makes no redundancy checks, so duplicate events are almost guaranteed.

### **TraceData**
Use `FDSNevt` to query FDSN servers for a single event's information and trace data.

In [ ]:
?FDSNevt

In [ ]:
ot = string(H[1].ot)

In [ ]:
# This is likely to throw many warnings
W = FDSNevt(ot, "UW.RCM..*, UW.RCS..*, CC.PANH..*, CC.OBSR..*", mag=mr, rad=rr)

#### **Saving Requests**
Request results can be saved to ASDF or written using `wseis` in low-level\
(SeisIO native) format.

##### ASDF
`write_hdf5` works with `SeisEvent` structures, just as with `SeisData`. For\
ASDF format, this also writes to, or appends, the "QuakeML" group within the\
volume.

In [ ]:
safe_rm(hdf_out)
write_hdf5(hdf_out, W)

\
**Warning**: due to the limitations of ASDF, picks and source-receiver\
geometry aren't written to file. Thus, `:hdr` and `:source` are preserved,\
but `:data` generally isn't.\
\
Thus, if we read in an event (here, using `read_asdf_evt`):

In [ ]:
W1.hdr

In [ ]:
W_arr = read_asdf_evt(hdf_out)
W1 = W_arr[1]
println(":hdr preserved = ", W1.hdr==W.hdr, "\n:source preserved = ", W1.source==W.source, "\n:data preserved = ", W1.data==W.data)

##### SeisIO low-level
`wseis` preserves everything:

In [ ]:
wseis("req.seis", W)
W1 = rseis("req.seis")[1]
W == W1

## E. **Cleanup**
Run these commands to delete save files when done:

In [ ]:
safe_rm("req.seis")
safe_rm(hdf_out)